In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-09-07T16:09:28.475671-07:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.14.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1029-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 256
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm


from time import time

In [4]:
%watermark --iversions

pandas: 1.5.3
numpy : 1.24.4



In [5]:
ID = 'id'
TARGET = 'target'

In [6]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [7]:
y_train = train[TARGET].ravel()

train.drop([ID, TARGET], axis=1, inplace=True)
test.drop([ID], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")

(595212, 57),(892816, 57)


In [8]:
%%time
kf = KFold(5, shuffle=True, random_state=137)

for i, (train_index, test_index) in enumerate(kf.split(train,y_train)):

        print(i)
        xgtrain, xgval = train.loc[train_index], train.loc[test_index]
        y_tr, y_val = y_train[train_index], y_train[test_index]

        y_train_5 = []
        for ii in range(5):
            y_train_5.append(y_tr)

        trains = []
        for ii in range(5):
            trains.append(xgtrain)

        y_train_5 = np.hstack(y_train_5)

        X_num_train_5 = pd.concat(trains, ignore_index=True, axis=0)

        columns = train.columns
        
        X_num_train_5['target'] = y_train_5
        X_num_train_5 = X_num_train_5.sample(frac=1).reset_index(drop=True)
        X_num_train_5 = X_num_train_5.sample(frac=1).reset_index(drop=True)
    
        xgval['target'] = y_val
        
        X_num_train_5.to_csv(f'../input/xgtrain_fold_{i}_5X.csv.gz', index=False, compression='gzip')
        xgval.to_csv(f'../input/xgval_fold_{i}_5X.csv.gz', index=False, compression='gzip')

0
1
2
3
4
CPU times: user 14min 1s, sys: 3.96 s, total: 14min 5s
Wall time: 14min 6s


In [10]:
X_num_train_5.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,target
0,1,1,7,0,1,1,0,0,0,0,...,0,0,9,1,0,1,0,1,0,0
1,0,1,5,0,6,1,0,0,0,0,...,1,3,10,0,1,0,0,0,0,0
2,0,2,1,0,0,0,0,1,0,0,...,2,5,9,0,0,1,0,1,0,0
3,3,1,4,0,0,1,0,0,0,0,...,1,7,2,0,0,1,1,1,0,0
4,1,2,4,0,0,0,0,0,1,0,...,2,1,4,0,1,1,1,1,0,0
